# Convolutional Neural Network Using Mel Spectrogram Classifying Gender

In [1]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import to_categorical
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
audio_data = np.load("../data/audio_data_mel_spec.npy")
labels = np.load("../data/wav_labels.npy")

# labels: modality-vocal channel-emotion-emotional intensity-statement-repetition-actor
# emotions: 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# odd number actors = male, even = female

# 1440 files: 24 speakers, 60 recordings per speaker
audio_data = audio_data.reshape(1440, 16384)

features = []

for i in range(1440):
    #female = 1
    #male = 0
    if (labels[i][6]%2 == 0):
        label = 1
    else:
        label = 0
    
    features.append([audio_data[i], label, labels[i][2]])

    
feature_df = pd.DataFrame(features, columns = ["mel_spec", "gender", "emotion"])

feature_df.head()

,mel_spec,gender,emotion
0,"[1.0769573011160105e-09, 2.8049595979240394e-1...",0,1.0
1,"[8.10519740213067e-09, 2.3336452770195137e-09,...",0,1.0
2,"[1.8761321030069666e-07, 2.7834460070153e-07, ...",0,1.0
3,"[1.213315243830948e-07, 1.1905444097237705e-07...",0,1.0
4,"[6.763266924281197e-08, 1.9553900187929685e-07...",0,2.0


In [3]:
## ===== Males ===== ##
X = np.array(feature_df.mel_spec.tolist())
y = np.array(feature_df.gender.tolist())

#20-80 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
X_train.shape

(1152, 16384)

In [4]:
# Reshape for CNN input
X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

# One-Hot encoding for classes
y_train = np.array(to_categorical(y_train, 2))
y_test = np.array(to_categorical(y_test, 2))

In [5]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

In [6]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train,
	y=y_train,
    epochs=40,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/40
9/9 [==============================] - 7s 784ms/step - loss: 1.5148 - accuracy: 0.5408 - val_loss: 0.7971 - val_accuracy: 0.5660
Epoch 2/40
9/9 [==============================] - 7s 761ms/step - loss: 1.0718 - accuracy: 0.5425 - val_loss: 0.7388 - val_accuracy: 0.6146
Epoch 3/40
9/9 [==============================] - 7s 733ms/step - loss: 0.9266 - accuracy: 0.6016 - val_loss: 0.6671 - val_accuracy: 0.6042
Epoch 4/40
9/9 [==============================] - 7s 736ms/step - loss: 0.7064 - accuracy: 0.5833 - val_loss: 0.6772 - val_accuracy: 0.6319
Epoch 5/40
9/9 [==============================] - 8s 903ms/step - loss: 0.6878 - accuracy: 0.5816 - val_loss: 0.6578 - val_accuracy: 0.6181
Epoch 6/40
9/9 [==============================] - 7s 740ms/step - loss: 0.6531 - accuracy: 0.5929 - val_loss: 0.6769 - val_accuracy: 0.6042
Epoch 7/40
9/9 [==============================] - 7s 730ms/step - loss: 0.6426 - accuracy: 0.6033 - val_loss: 0.6974 - val_accuracy: 0.6007
Epoch 8/40
9/9 [====